In [1]:
import json
import numpy as np
import pandas as pd
from scipy.stats.mstats import gmean

In [2]:
df = pd.read_csv('../output/primary_data_preprocessed.csv')
df = df.set_index(['country', 'primary_index', 'decile'])

In [3]:
def weighted_geometric_mean(data: list[tuple[pd.Series | pd.DataFrame, float]]):
    # computed in log space for numerical stability
    wsum = 0
    logsum = 0
    for values, weight in data:
        if weight != 0:
            logsum += weight * np.log(values)
            wsum += weight
    
    return np.exp(logsum / wsum)

In [4]:
with open('../data/ewbi_indicators.json') as f:
    config = json.load(f)['EWBI']

In [6]:
all_codes = set()
all_secondaries = set()
for prio in config:
    print(prio['name'])
    for component in prio['components']:
        print('\t', component['name'])
        print(f'\t\t {component['name']}: {component['weight']}')
        all_secondaries.add(component['name'])
        for ind in component['indicators']:
            if ind['code']:
                print(f'\t\t\t {ind['code']}: {ind['weight']}')
                all_codes.add(ind['code'])
    print()

Agriculture and Food
	 Nutrition need
		 Nutrition need: 2/3
			 AN-SILC-1: 1
			 AN-EHIS-1: 2
	 Nutrition expense
		 Nutrition expense: 1/3
			 AE-HBS-1: 1
			 AE-HBS-2: 1
			 AE-EHIS-1: 2

Energy and Housing
	 Housing quality
		 Housing quality: 1/3
			 HQ-SILC-1: 2
			 HQ-SILC-2: 2
	 Energy
		 Energy: 1/3
			 HE-SILC-1: 2
			 HE-SILC-2: 1
	 Housing expense
		 Housing expense: 1/3
			 HH-SILC-1: 1
			 HH-HBS-1: 1
			 HH-HBS-2: 1
			 HH-HBS-3: 1
			 HH-HBS-4: 1

Equality
	 Life satisfaction
		 Life satisfaction: 1/4
			 EL-SILC-1: 2
			 EL-EHIS-1: 2
	 Security
		 Security: 1/4
			 ES-SILC-1: 1
			 ES-SILC-2: 1
	 Community
		 Community: 1/4
			 EC-SILC-1: 2
			 EC-SILC-2: 2
			 EC-HBS-1: 1
			 EC-HBS-2: 1
			 EC-EHIS-1: 2
	 Digital Skills
		 Digital Skills: 1/4
			 ED-EHIS-1: 2

Health and Animal Welfare
	 Health condition and impact
		 Health condition and impact: 1/2
			 AH-SILC-1: 2
			 AH-SILC-2: 2
			 AH-SILC-3: 2
			 AH-SILC-4: 2
			 AH-EHIS-1: 2
			 AH-EHIS-2: 2
	 Health cost an

In [7]:
print("Present in json file but not in index:", all_codes.difference(df.index.get_level_values('primary_index')))
print("Present in index but not in json file:", df.index.get_level_values('primary_index').difference(all_codes))

Present in json file but not in index: {'RU-LFS-1', 'IS-SILC-1', 'IS-SILC-2'}
Present in index but not in json file: Index([], dtype='object', name='primary_index')


In [8]:
# shouldn't be neeed in the future as preprocess shouldn't contain any NaN in the future
# but for now with some bugs it does du to some indicators being almost always 0
# in the meantime, we replace by a default value of 0.5
df = df.fillna(0.5)

In [ ]:
secondary = {}
# separate countries as indicators aren't all available for all countries
for country, cdf in df.groupby('country'):
    #print("EL-SILC-1" in cdf.loc[country].index)
    #print(cdf.loc[country, 'EL-SILC-1'])
    cdf = cdf.loc[country]
    for prio in config:
        for component in prio['components']:
            logsum = 0
            wsum = 0
            for ind in component['indicators']:
                code = ind['code']
                weight = ind['weight']
                if code in cdf.index and weight != 0:
                    logsum += np.log(cdf.loc[code])
                    wsum += weight
            if wsum == 0:
                print('Missing', country, component['name'])
            else:
                secondary[country, prio['name'], component['name']] = np.exp(logsum / wsum)

secondary = pd.concat(secondary, names=('country', 'eu_priority', 'secondary_indicator'))

Missing CH Nutrition expense
Missing CH Digital Skills
Missing CH Accidents and addictive behaviour
Missing CH Education expense
Missing CZ Education
Missing EA Nutrition need
Missing EA Nutrition expense
Missing EA Housing quality
Missing EA Energy
Missing EA Housing expense
Missing EA Life satisfaction
Missing EA Security
Missing EA Community
Missing EA Digital Skills
Missing EA Health condition and impact
Missing EA Health cost and medical care
Missing EA Accidents and addictive behaviour
Missing EA Education
Missing EA Education expense
Missing EA Leisure and culture
Missing EA Type of job and market participation
Missing EA Unemployment
Missing EA Tourism
Missing ES Digital Skills
Missing ES Accidents and addictive behaviour
Missing EU Nutrition need
Missing EU Nutrition expense
Missing EU Housing quality
Missing EU Energy
Missing EU Housing expense
Missing EU Life satisfaction
Missing EU Security
Missing EU Community
Missing EU Digital Skills
Missing EU Health condition and impac

In [42]:
secondary.to_csv('../output/secondary_indicators.csv')

In [43]:
print(all_secondaries.difference(secondary.index.get_level_values('secondary_indicator')))
print(secondary.index.get_level_values('secondary_indicator').difference(all_secondaries))

set()
Index([], dtype='object', name='secondary_indicator')


In [46]:
secondary

2004  \
country eu_priority                       secondary_indicator decile             
AT      Agriculture and Food              Nutrition need      1       0.580133   
                                                              2       0.554855   
                                                              3       0.484119   
                                                              4       0.492833   
                                                              5       0.483813   
...                                                                        ...   
UK      Sustainable Transport and Tourism Tourism             6       0.724993   
                                                              7       0.745031   
                                                              8       0.760814   
                                                              9       0.769893   
                                                              10      0.771329   

                                                                          2005  \
country eu_priority                       secondary_indicator decile             
AT      Agriculture and Food              Nutrition need      1       0.596725   
                                                              2       0.505512   
                                                              3       0.452037   
                                                              4       0.447097   
                                                              5       0.473257   
...                                                                        ...   
UK      Sustainable Transport and Tourism Tourism             6       0.724081   
                                                              7       0.740952   
                                                              8       0.758836   
                                                              9       0.764236   
                                                              10      0.770650   

                                                                          2006  \
country eu_priority                       secondary_indicator decile             
AT      Agriculture and Food              Nutrition need      1       0.572088   
                                                              2       0.541583   
                                                              3       0.500104   
                                                              4       0.443149   
                                                              5       0.477752   
...                                                                        ...   
UK      Sustainable Transport and Tourism Tourism             6       0.744643   
                                                              7       0.741053   
                                                              8       0.748799   
                                                              9       0.766855   
                                                              10      0.775567   

                                                                          2007  \
country eu_priority                       secondary_indicator decile             
AT      Agriculture and Food              Nutrition need      1       0.619895   
                                                              2       0.524952   
                                                              3       0.492230   
                                                              4       0.480092   
                                                              5       0.449609   
...                                                                        ...   
UK      Sustainable Transport and Tourism Tourism             6       0.747607   
                                                              7       0.745918   
                                                     

In [57]:
priorities = {}
for country, cdf in secondary.groupby('country'):
    cdf = cdf.loc[country]
    for prio in config:
        pname = prio['name']
        if pname in cdf.index:
            cpdf = cdf.loc[pname]
            logsum = 0
            wsum = 0
            for c in prio['components']:
                name = c['name']
                weight = c['weight']
                try:
                    weight = float(weight)
                except ValueError:
                    numerator, denominator = map(int, weight.split('/'))
                    weight = float(numerator) / denominator

                if name in cpdf.index and weight != 0:
                    logsum += np.log(cpdf.loc[name])
                    wsum += weight

            if wsum == 0:
                print('Missing', country, pname)
            else:
                priorities[country, pname] = np.exp(logsum / wsum)

priorities = pd.concat(priorities, names=['country', 'eu_priority'])

In [60]:
priorities.to_csv('../output/eu_priorities.csv')

In [68]:
ewbi = {}
for country, cdf in priorities.groupby('country'):
    cdf = cdf.loc[country]
    factors = [(cdf.loc[prio], 1)  for prio in cdf.index.get_level_values('eu_priority')]
    ewbi[country] = weighted_geometric_mean(factors)
    
ewbi = pd.concat(ewbi, names=['country'])

In [71]:
ewbi.to_csv('../output/ewbi_results.csv')